# Задание 2

In [75]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [76]:
%%time
# распарсим "Выстрел" и запишем в json
from pymystem3 import Mystem
import json

with open('Выстрел.txt') as f:
    text = f.read()
m = Mystem()
with open('mystem_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(m.analyze(text), f, ensure_ascii=False, indent=4)

Wall time: 1min 40s


In [2]:
%%time
# токенизируем текст с помощью nltk и анализируем с помощью pymorphy
from nltk import word_tokenize
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json

with open('Выстрел.txt') as f:
    text = f.read()
    text = [x.strip(punctuation).lower() for x in word_tokenize(text) if x.isalpha()]

morph = MorphAnalyzer()
pymorphy_analysis = []
for word in text:
    word_analysis = {}
    ana = morph.parse(word)[0]
    word_analysis['lemma'] = ana.normal_form
    word_analysis['part_of_speech'] = ana.tag.POS
    pymorphy_analysis.append(word_analysis)

with open('pymorphy_analysis.json', 'w', encoding='utf-8') as f:
    json.dump(pymorphy_analysis, f, ensure_ascii=False, indent=4)

Wall time: 738 ms


In [2]:
# находим долю каждой из частей речи в тексте
import json
from collections import Counter

with open('pymorphy_analysis.json', encoding='utf-8') as f:
    pymorphy_analysis = json.loads(f.read())

parts_of_speech = {}
for lemma in pymorphy_analysis:
    if lemma['part_of_speech'] not in parts_of_speech:
        parts_of_speech[lemma['part_of_speech']] = []
    parts_of_speech[lemma['part_of_speech']].append(lemma['lemma'])
pos_statistics = {}
for pos in parts_of_speech:
    pos_statistics[pos] = len(parts_of_speech[pos])
pos_statistics = list(pos_statistics.items())
pos_statistics.sort(key=lambda i: i[1], reverse=True)
print('Доля каждой части речи:')
for pos in pos_statistics:
    print(pos[0], round(pos[1]/len(pymorphy_analysis)*100, 2), end='')
    print('%\n')


# находим топ-20 глаголов и наречий
print()
print('Топ-20 глаголов: ', ', '.join([word[0] for word in Counter(parts_of_speech['VERB']).most_common(20)]))
print('Топ-20 наречий: ', ', '.join([word[0] for word in Counter(parts_of_speech['ADVB']).most_common(20)]))

Доля каждой части речи:
NOUN 23.54%

VERB 15.8%

NPRO 13.56%

ADJF 9.9%

PREP 9.81%

CONJ 8.34%

ADVB 5.24%

PRCL 4.91%

INFN 2.91%

ADJS 1.5%

GRND 1.14%

PRTF 1.0%

NUMR 0.89%

PRTS 0.5%

PRED 0.28%

INTJ 0.25%

COMP 0.25%

None 0.19%


Топ-20 глаголов:  быть, сказать, стать, знать, отвечать, продолжать, мочь, дать, иметь, сесть, ждать, встать, выйти, хотеть, увидеть, случаться, взять, видеть, бывать, вынуть
Топ-20 наречий:  уже, никогда, однажды, очень, наконец, тут, где, почти, вечером, пешком, назад, уж, снова, несколько, чрезвычайно, обыкновенно, прежде, столь, скоро, поминутно


In [22]:
# составим топ-25 биграмм и триграмм
import json
import nltk
from collections import Counter

with open('pymorphy_analysis.json', encoding='utf-8') as f:
    text = json.loads(f.read())

text = [word['lemma'] for word in text]
bigrams = nltk.bigrams(text)
trigrams = nltk.trigrams(text)
print('Топ-25 биграмм:')
print(*[' '.join(bigram[0]) for bigram in Counter(bigrams).most_common(25)], sep='\n')
print()
print('Топ-25 триграмм:')
print(*[' '.join(trigram[0]) for trigram in Counter(trigrams).most_common(25)], sep='\n')

'''
Появление в списке сочетаний с отрицанием ("я не", "не быть"), личными местоимениями 1-го и 3-го лица ("я быть", "он не"), предлогами ("я в", "у он"),а также всевозможных их комбинаций объяснимо: все эти лексемы находятся вверху списка частотных лексем для русского языка в целом.
Появление сочетаний "наш полка", "в наш полка" объясняются тематикой повести "Выстрел". Сочетания "ваш сиятельство", "ваш сиятельство не", "сильвио взять мел" отражают важных персонажей повести.
"по крайний мера" ("по крайней мере"), "год тот назад" ("год тому назад") -- частотные сочетания сами по себе.
'''

Топ-25 биграмм:
я не
я в
ваш сиятельство
он быть
с он
не быть
в наш
и в
с я
я быть
я с
сказать он
не знать
у он
он я
что я
быть в
наш полка
никогда не
между мы
он не
и не
он в
у я
не иметь

Топ-25 триграмм:
в наш полка
у я в
по крайний мера
ваш сиятельство не
никто не знать
выйти в отставка
в отставка и
поселиться в бедный
два или три
он драться отвечать
сильвио взять мел
взять мел и
мы послать к
на он глядеть
с тот пора
я с он
обыкновенно тут же
что я не
быть в мой
в мой рука
год тот назад
тот назад я
и дать я
молодая и прекрасный
стать ходить взад


'\nПоявление в списке сочетаний с отрицанием ("я не", "не быть"), личными местоимениями 1-го и 3-го лица ("я быть", "он не"), предлогами ("я в", "у он"),а также всевозможных их комбинаций объяснимо: все эти лексемы находятся вверху списка частотных лексем для русского языка в целом.\nПоявление сочетаний "наш полка", "в наш полка" объясняются тематикой повести "Выстрел". Сочетания "ваш сиятельство", "ваш сиятельство не", "сильвио взять мел" отражают важных персонажей повести.\n"по крайний мера" ("по крайней мере"), "год тот назад" ("год тому назад") -- частотные сочетания сами по себе.\n'